In [ ]:
import functools
import numpy as np
from scipy.linalg import expm

import qiskit as qk
import qiskit_dynamics as qk_d
import qiskit.providers.fake_provider as qk_fp

import qutip as qt
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import importlib

# Setting up the Problem
-----

We would like to solve the following equation,
\begin{align}
i\frac{\partial}{\partial t}|\psi(t)\rangle=\hat{H}|\psi(t)\rangle
\end{align}
where, $\hat{H}$ describes the Hamiltonian of the Ising model,
\begin{align}
\hat{H}=\frac{1}{2}\sum_{i=0}^{N-2}J_x\sigma^x_i \sigma^x_{i+1}+\frac{1}{2}\sum_{i=0}^{N-1}h_{z}\sigma^z_{i}
\end{align}
This Hamiltonian describes a magnetic chain (or wire) consisting of $N$ magnetic atoms, where the magnetic atoms are arranged in a line. Each atom interacts with its nearest neighboring atom with the strength of the coupling given by $(J_x,J_y,J_z)$ in addition to an external magnetic field given by the vector $\vec{h}=(hx,hy,hz)$. For simplicity, consider the following values for the parameters,
\begin{align}
(J_x= 2\pi ) \textrm{ and } (hz=2\pi)
\end{align}

We also set the initial state to be have all spins in the $|+\rangle$ state,
\begin{align}
|\psi(t=0)\rangle&=|+\rangle^{\otimes N}\\
&=\frac{1}{2^{N/2}}(|0\rangle+|1\rangle)\otimes(|0\rangle+|1\rangle)\otimes\cdots N\textrm{ times}
\end{align}



In particular, we will be interested in the time-dependence of the following quantity,
\begin{align}
\frac{1}{N}\left\langle \sum_{i=0}^{N-1} \sigma^z_i(t) \right\rangle=\frac{1}{N}\sum_i \langle\psi(t)| \sigma^z_i |\psi(t)\rangle
\end{align}

In [ ]:
# Set the parameters globally for the model
N = 4  # number of spins
hz = 1.0 * 2 * np.pi  # magnetic field along z
Jx = 1.0 * 2 * np.pi  # Coupling along x
Δt = 0.05  # time step for integration
tlist = np.arange(50) * Δt  # time values

# Global backend
backend = qk_fp.FakeManila()

Now we are ready to create the Unitary operator for the Ising Hamiltonian by composing one and two qubit gates. We show the first and second order Trotter decomposition where, for $\hat{H}=\hat{A}+\hat{B}$, 
\begin{align}
e^{-i\hat{H}\Delta t}&\approx e^{-i\hat{A}\Delta t}e^{-i\hat{B}\Delta t} + O(\Delta t^2)\textrm{ (First Order Trotter)}\\
e^{-i\hat{H}\Delta t}&\approx e^{-i\hat{B}\frac{\Delta t}{2}} e^{-i\hat{A}\Delta t}e^{-i\hat{B}\frac{\Delta t}{2}}+O(\Delta t^3) \textrm{ (Second Order Trotter)}
\end{align}

In [ ]:
def create_initial_state(qr, cr):
    '''
    Creates the initial state for the experiment.
    Here we would like to prepare the |+>|+>|+>|+> state.
    '''
    circ = qk.QuantumCircuit(qr, cr)
    [circ.h(qr[i]) for i in range(qr.size)]
    return circ


def rxx(circ, θx, q1, q2):
    '''
    Implements the exp(-i theta/2 sx_1 sx_2) gate.
    '''
    circ.cx(q1, q2)
    circ.rx(θx, q1)
    circ.cx(q1, q2)
    return circ


def first_order_Trotter_unitary(circ, Δt, barrier=False):
    '''
    Applies a Unitary for a single time step U(Δt) using first-order trotter expansion to the input quantum circuit.
    input : circ is a quantum circuit
    output : circ-U(Δt)-
    '''
    # Layer A: coupling XX between n and n+1
    # Induce parallel CR gates
    if barrier:
        circ.barrier()
    for p in range(0, circ.num_qubits - 1, 2):
        circ = rxx(circ, Jx * Δt, circ.qubits[p], circ.qubits[p + 1])
    if barrier:
        circ.barrier()
    for p in range(1, circ.num_qubits - 1, 2):
        circ = rxx(circ, Jx * Δt, circ.qubits[p], circ.qubits[p + 1])

    #  Alt., a naive loop will apply XX rotations sequentially, failing to stack.
    # for p in range(0, circ.num_qubits - 1):
    #     circ = rxx(circ, Jx * Δt, circ.qubits[p], circ.qubits[p + 1])

    # Layer B: on-site Z
    for p in range(circ.num_qubits):    
        circ.rz(hz*Δt, circ.qubits[p])

    return circ


def second_order_Trotter_unitary(circ, Δt):
    '''
    Applies a Unitary for a single time step U(Δt) using second-order trotter expansion to the input quantum circuit.
    input : circ is a quantum circuit
    output : circ-U(Δt)-
    '''
    # Layer 2: on-site Z (half)
    for p in range(circ.num_qubits):
        circ.rz(hz * Δt / 2, circ.qubits[p])
 
    # Layer 1: coupling XX between n and n+1
    # Induce parallel structure (and symmetry)
    for p in range(1, circ.num_qubits - 1, 2):
        circ = rxx(circ, Jx * Δt / 2, circ.qubits[p], circ.qubits[p+1])
    # NOTE: Add barriers to separate the parallel section
    circ.barrier()
    for p in range(0, circ.num_qubits - 1, 2):
        circ = rxx(circ, Jx * Δt, circ.qubits[p], circ.qubits[p+1])
    circ.barrier()
    for p in range(1, circ.num_qubits - 1, 2):
        circ = rxx(circ, Jx * Δt / 2, circ.qubits[p], circ.qubits[p+1])

    # Layer 2: on-site Z (half)
    for p in range(circ.num_qubits):
        circ.rz(hz * Δt / 2, circ.qubits[p])
    return circ

In [ ]:
qr = qk.QuantumRegister(N)
cr = qk.ClassicalRegister(N)
circ = create_initial_state(qr, cr)
circ.barrier()
circ = first_order_Trotter_unitary(circ, Δt)
circ.barrier()
# circ.measure(qr, cr)
circ.draw('mpl')

# Transpiler & scheduler
-----

Make a pass to get gate times.

In [ ]:
# transp_circ = qk.transpiler.passes.RemoveBarriers()(circ)
basis_gates = ['rz','x','sx', 'ecr', 'id']
transp_circ = circ
transp_circ  = qk.transpiler.passes.RemoveBarriers()(transp_circ)
transp_circ = qk.transpile(
    transp_circ,
    backend,
    basis_gates=basis_gates,
    optimization_level=1
)
transp_circ  = qk.transpiler.passes.RemoveBarriers()(transp_circ)
# transp_circ.draw('mpl')

ns = 1e-9
instruction_durations = qk.transpiler.InstructionDurations(
    [('rz', None, 0 * ns, 's'),
    ('x', None, 30 * ns, 's'),
    ('id', None, 30 * ns, 's'),
    ('ecr', None, 150 * ns, 's'),
    ('sx', None, 30 * ns, 's'),
    ("reset", None, 1500 * ns, 's'),
    ("measure", None, 1500 * ns, 's')],
    dt = ns
)

# dd_sequence = [qk.circuit.library.XGate(), qk.circuit.library.XGate()]
pm = qk.transpiler.PassManager(
    [qk.transpiler.passes.ASAPScheduleAnalysis(instruction_durations)],
    #  qk.transpiler.passes.PadDynamicalDecoupling(instruction_durations, dd_sequence)]
)
pm_circ = pm.run(transp_circ)
pm_circ.duration = 1_000 # dt

qk.visualization.timeline.draw(
    pm_circ,
    # style=qk.visualization.timeline.IQXSimple()
)

Compare to the actual qiskit schedule.

In [ ]:
sched_circ = qk.schedule(qk.transpile(circ, backend), backend, method='alap')
# sched_circ.draw()
sched_circ.filter(channels=[qk.pulse.DriveChannel(i) for i in range(5)] + [qk.pulse.ControlChannel(i) for i in range(5)]).draw()

# Simulator
-----

- Gates are applied to qubit registers. 
- A qubit register is attached to a circuit. 
- Each gate has a pulse implementation which it applies to a qubit or pair of qubits using 
a qubit model(s).
- We construct models from qubit registers and attach pulses to register
channels. 
- Virtual Z's are applied to set the correct carrier offset as a separate
 operation.

In [ ]:
# configure jax to use 64 bit mode
import jax
jax.config.update("jax_enable_x64", True)

# tell JAX we are using CPU
jax.config.update('jax_platform_name', 'cpu')

# set default backend
qk_d.array.Array.set_default_backend('jax')
qk_d.array.Array.default_backend()

In [ ]:
import sys
sys.path.append("../")
import pulse_simulator as ps

In [ ]:
importlib.reload(ps)

## Device set up

In [ ]:
# Undo units
GHz = 1e-9
ns = 1e9

# Sample rate of the backend in ns.
dt = 1 / 4.5  # ns
duration = 48.9  # ns

In [ ]:
# Initialize device
# =====
registers = [0, 1, 2, 3]  # Active registers
configuration = backend.configuration()
properties = backend.properties()

qubit_properties = {
    i: {
        "freq": properties.frequency(i) * GHz,
        "rabi_freq": 0.1,  # GHz,
        "t1": properties.t1(i) * ns,
        "t2": properties.t2(i) * ns,
    } for i in registers
}

cr_properties = {
    "coupling": 10e6 * GHz,
    "alpha": -100e6 * GHz  # for all qubits (not treated as transmons)
}

The next cell defines a toy pulse and the crosstalk coupling constant using the backend
parameters.

In [ ]:
def gaussian_envelope(qubit_props, index):
    """ Define gaussian envelope function to approximately implement an sx 
    gate.

    NOTE: this is just a placeholder

    Arguments:
        qubit_props -- Qubit properties
        index -- Index of desired qubit

    Returns:
        Qiskit pulse implementing an sx gate
    """
    rabi_freq = qubit_props[index]["rabi_freq"]
    amplitude = 1. / 1.75
    sigma = 0.6985 / rabi_freq / amplitude
    duration = 4 * sigma
    steps = int(duration / dt)
    beta = 2.0
    return qk.pulse.Gaussian(steps, amplitude, sigma / dt, beta)


def zz_coupling(edge, qubit_props, cr_props):
    """ Return the crosstalk coupling amount.

    Arguments:
        edge -- tuple
        qubit_props -- Qubit properties
        cr_props -- Cross resonance gate properties

    Returns:
        _description_
    """
    i1, i2 = edge
    alpha = cr_props["alpha"]
    J = cr_props["coupling"] 
    w1 = qubit_props[i1]["freq"]
    w2 = qubit_props[i2]["freq"]
    delta_2 = (w1 - w2)**2
    return - 2 * np.pi * J**2 * alpha / (alpha**2 - delta_2)

In [ ]:
crosstalk_graph = {
    edge: zz_coupling(edge, qubit_properties, cr_properties) 
    for edge in ps.get_edges(backend) 
    if edge[0] in registers and edge[1] in registers
}
crosstalk_graph

## Single qubit moment

In [ ]:
# Choose gates from the circuit
# ======
virtual_zs = {
    0: np.pi / 2,
    1: 0.,
    2: np.pi / 2,
    3: 0.
}

gates = {
    0: "sx_red",
    1: "sx_red",
    2: "sx_red",
    3: "sx_red"
}

# Lookup table
# =====
# should also include "x_red", "sx_blue", "x_blue"
gate_lookup = {
    "sx_red": gaussian_envelope(qubit_properties, 0)
}

# Design pulse schedule
# =====
with qk.pulse.build(name="Current moment") as pulse_moment:
    for i, gate in gates.items():
        channel = qk.pulse.DriveChannel(i)
        qk.pulse.shift_phase(virtual_zs[i], channel)
        qk.pulse.play(gate_lookup[gate], channel)


# Define carriers (if RWA)
# NOTE: Unsure if we want to do this yet
# =====
drive_carriers = {f"d{r}": qubit_properties[r]["freq"] for r in registers}

In [ ]:
# Create a system model
# =====
# Control model
#   Single qubit circuit moment: attach RX models to each qubit
H_drift = 0.
Hs_control = []
Hs_channels = []
As_static = []
for (qubit_index), label in gates.items():
    qubit = transp_circ.qubits[qubit_index]
    # if qubit_index in registers:
    Hj_drift, Hjs_control, Hjs_channel = ps.rx_model(
        qubit,
        transp_circ,
        **qubit_properties[qubit_index]
    )
    # NOTE: doesn't work with quantum channel simulator yet
    Ajs_static = ps.qubit_decay_model(
        qubit,
        transp_circ,
        **qubit_properties[qubit_index]
    )
    H_drift += Hj_drift
    Hs_control += Hjs_control
    Hs_channels += Hjs_channel
    As_static += Ajs_static


# Crostalk
#   ZZ coupling across pairs
H_crosstalk = ps.crosstalk_model(transp_circ, crosstalk_graph)

    
# Construct the solver
# =====
solver = qk_d.Solver(
    static_hamiltonian=H_drift, # + H_crosstalk,
    hamiltonian_operators=Hs_control,
    static_dissipators=None, # As_static if len(As_static) > 0 else None,
    rotating_frame=H_drift,
    rwa_cutoff_freq=2 * min(drive_carriers.values()),
    hamiltonian_channels=Hs_channels,
    channel_carrier_freqs=drive_carriers,
    dt=dt
)

Solve three ways
1. State vector sim (sparse)
2. Unitary sim
3. Channel sim (not working)

- The job of constructing and running this circuit moment is to give you a final unitary at the end.
- If there is dissipation, you will need a final channel. This requires a superoperator representation of the initial unitiary. The channel can then be returned. This turns out to be very slow. How can I fix it?
- We will combine these final unitaries or channels into a list of moments. We will combine those moments to achieve a simulation. This should be possible in Qiskit using the evolve functionality, even if we have different representations for each moment (channel, unitary).

In [ ]:
# Start the qubit in its ground state.
y0 = qk.quantum_info.states.Statevector(
    functools.reduce(
        np.kron, 
        np.repeat([[1, 0]], len(transp_circ.qubits), axis=0)
    )
)

# Identity matrix
id_label = ''.join(['I'] * len(transp_circ.qubits))
U0 = qk.quantum_info.Operator.from_label(id_label)

# NOTE: Unsure if correct
# Identity channel
C0 = qk.quantum_info.SuperOp(U0)

In [ ]:
# Unitary or channel sim.
# NOTE: Bug when solving vectorized problems
if False: #len(As_static) > 0:
    solver.model.evaluation_mode = 'sparse_vectorized'
    sol = solver.solve(
        t_span=[0., duration], 
        y0=C0, 
        signals=pulse_moment,
        atol=1e-8,
        rtol=1e-8,
        method='jax_odeint'
    )

else:
    solver.model.evaluation_mode = 'sparse'
    sol = solver.solve(
        t_span=[0., duration],
        y0=U0, 
        signals=pulse_moment,
        atol=1e-8, 
        rtol=1e-8,
        method='jax_odeint'
    )

# Sparse state vector sim
solver.model.evaluation_mode = 'sparse'

sol1 = solver.solve(
    t_span=[0., duration],
    y0=y0,
    signals=pulse_moment,
    atol=1e-8,
    rtol=1e-8,
    method='jax_odeint'
)


In [ ]:
basis = ps.hilbert_space_basis([2] * len(transp_circ.qubits))

# Check final states
yf1 = sol1.y[-1]
y_after_1qb_moment = yf1

Uf = sol.y[-1]
yf = y0.evolve(Uf)

# Compare
print(f"Are equal? {yf1 == yf}\n")
print(f"Are close? ||y1 - y2|| = {np.linalg.norm(yf1 - yf)}\n")

# States
ps.print_wavefunction(yf, basis)
print()

# ps.print_wavefunction(yf1, basis)

In [ ]:
fig, ax = plt.subplots(figsize=[2,2])
ax.axis('off')
ax.imshow(np.abs(Uf))

### Aside: Plot pulses

In [ ]:
"""
- Signals are ordered by the channels of the pulse schedule.
- From the documentation: "Instances of ScheduleBlock must first be converted 
to Schedule using the block_to_schedule() function in Qiskit Pulse." This can
be accomplished with `qk.pulse.transforms.block_to_schedule(moment)`.

I didn't see that begin necessary currently.
""";

converter = qk_d.pulse.InstructionToSignals(dt, carriers=drive_carriers)
signals = converter.get_signals(pulse_moment)

# TODO: probably a better way to set duration

fig, ax = ps.plot_pulse_schedule(
    signals, 
    pulse_moment, 
    duration=pulse_moment.duration / 4,
    figsize=[8, 6]
)
fig.tight_layout()

## Two qubit moment

In [ ]:
def cr_kwargs(control_index, target_index, qubit_props, cr_props):
    c_props = qubit_props[control_index]
    t_props = qubit_props[target_index]
    d = {
        "freq_c": c_props["freq"],
        "freq_t": t_props["freq"],
        "rabi_freq_c": c_props["rabi_freq"],
        "rabi_freq_t": t_props["rabi_freq"],
        "coupling": cr_props["coupling"],
        "alpha": cr_props["alpha"],
    }
    return d